# Compare NLP Techniques: Build Model On doc2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('../../../data/X_train.csv')
X_test = pd.read_csv('../../../data/X_test.csv')
y_train = pd.read_csv('../../../data/y_train.csv')
y_test = pd.read_csv('../../../data/y_test.csv')

### Create doc2vec Vectors

In [2]:
# Created TaggedDocument vectors for each text message in the training and test sets
tagged_docs_train = [gensim.models.doc2vec.TaggedDocument(v, [i])
                     for i, v in enumerate(X_train['clean_text'])]
tagged_docs_test = [gensim.models.doc2vec.TaggedDocument(v, [i])
                    for i, v in enumerate(X_test['clean_text'])]

In [3]:
# What do these TaggedDocument objects look like?
tagged_docs_train[:10]

[TaggedDocument(words="['ill', 'see', 'prolly', 'yeah']", tags=[0]),
 TaggedDocument(words="['hi', 'darlin', 'ive', 'got', 'back', 'really', 'nice', 'night', 'thanks', 'much', 'lift', 'see', 'u', 'tomorrow', 'xxx']", tags=[1]),
 TaggedDocument(words="['wait', '4', 'u', 'lor', 'need', '2', 'feel', 'bad', 'lar']", tags=[2]),
 TaggedDocument(words="['dont', 'know', 'u', 'u', 'dont', 'know', 'send', 'chat', '86688', 'lets', 'find', '150pmsg', 'rcvd', 'hgsuite3422landsroww1j6hl', 'ldn', '18', 'years']", tags=[3]),
 TaggedDocument(words="['dont', 'use', 'hook']", tags=[4]),
 TaggedDocument(words="['feel', 'always', 'happy', 'slowly', 'becomes', 'habit', 'amp', 'finally', 'becomes', 'part', 'life', 'follow', 'happy', 'morning', 'amp', 'happy', 'day']", tags=[5]),
 TaggedDocument(words="['ur', 'face', 'test', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ltgt', 'select', 'number', 'tell', 'ur', 'face', 'astrology', 'waiting', 'quick', 'reply']", tags=[6]),
 TaggedDocument(words="['well', 'done

In [4]:
# Train a basic doc2vec model
d2v_model = gensim.models.Doc2Vec(tagged_docs_train,
                                 vector_size=100,
                                 window=5,
                                 min_count=2)

In [5]:
# Infer the vectors to be used in training and testing
train_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_train]
test_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_test]  

### Fit RandomForestClassifier On Top Of Document Vectors

In [6]:
# Fit a basic model, make predictions on the holdout test set, and the generate the evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

rf = RandomForestClassifier()
rf_model = rf.fit(train_vectors, y_train.values.ravel())

y_pred = rf_model.predict(test_vectors)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.887 / Recall: 0.39 / Accuracy: 0.917
